In [ ]:
import json, os, zipfile, numpy as np
from scipy.ndimage import zoom
import logging
logging.basicConfig(level=logging.INFO)
P = '/kaggle/input/google-code-golf-2025'

def resize(g, t): return np.clip(zoom(g, (t[0]/g.shape[0], t[1]/g.shape[1]), order=0)[:t[0], :t[1]], 0, 9).astype(int)
def rot(g, k=1): return np.rot90(g, k)
def flip(g, a=0): return np.flip(g, a)
def extend(g): return np.pad(g, ((0, 2), (0, 2)), mode='edge')
def score(g, o): return 1 - np.mean(np.abs(g - o)) / 9 if g.shape == o.shape else 0
def infer_shape(ti, to): return tuple(np.mean([o.shape for o in to], 0).astype(int)) if to else ti.shape

def make_py(task_id, tr, to):
    ts = infer_shape(tr[0], to)
    best_score, pred_func = 0, lambda g: resize(g, ts)
    for func, param, name in [(rot, 1, 'np.rot90(g,1)'), (rot, 2, 'np.rot90(g,2)'), (rot, 3, 'np.rot90(g,3)'), (flip, 0, 'np.flip(g,0)'), (flip, 1, 'np.flip(g,1)'), (extend, 0, 'np.pad(g,((0,2),(0,2)),\'edge\')'), (lambda x: resize(x, ts), 0, f'np.clip(zoom(g,({ts[0]/tr[0].shape[0]},{ts[1]/tr[0].shape[1]}),order=0)[:{ts[0]},:{ts[1]}],0,9).astype(int)')]:
        try:
            q = func(tr[0], param) if param else func(tr[0])
            if q.shape == ts and score(q, to[0]) > best_score:
                best_score = score(q, to[0])
                pred_func = lambda g, f=func, p=param: f(g, p) if p else f(g)
                py_code = f"import json,numpy as np;from scipy.ndimage import zoom;g=np.array(json.loads(input()))['input'];print(json.dumps({{'output':{name}.tolist()}}))"
        except: pass
    logging.info(f"Task {task_id}: Score {best_score:.3f}")
    return py_code

def main():
    tasks = [t for t in os.listdir(P) if t.startswith('task') and t.endswith('.json')]
    z = zipfile.ZipFile('submission.zip', 'w')
    for t in tasks:
        task_id = t.split('.')[0]
        logging.info(f"Processing {task_id}")
        with open(os.path.join(P, t)) as f: task = json.load(f)
        tr = [np.array(d['input']) for d in task['train']]
        to = [np.array(d['output']) for d in task['train']]
        py = make_py(task_id, tr, to)
        z.writestr(f"{task_id}.py", py)
    z.close()
    logging.info("submission.zip created!")

if __name__ == "__main__":
    try: main()
    except Exception as e: logging.error(f"Error: {str(e)}")